ΕΠΙΧΕΙΡΗΣΙΑΚΉ ΈΡΕΥΝΑ 

ΕΡΓΑΣΊΑ 8ΟΥ ΕΞΑΜΉΝΟΥ 2024

ΧΡΙΣΤΊΝΑ ΤΣΑΡΆΒΑ

ΑΕΜ: 10592

ctsarava@ece.auth.gr

<h1><u>1st Problem: Scheduling Flight Landings</h1></u>

<h3><u>Setting Up Python Environment</h3></u>

To optimize the airplane landing schedule and present the results neatly, we use Python libraries `gurobipy` and `tabulate`. First, we install and import these libraries.

In [105]:
# Import necessary libraries
import gurobipy as gp
from gurobipy import GRB
from tabulate import tabulate

# Install necessary libraries (for Jupyter notebook)
%pip install gurobipy
%pip install tabulate

<h3><u>Defining Input Parameters</h3></u>

This section initializes the input data as specified in the problem statement, including arrival times and penalties for each aircraft.


In [106]:
# Constants
LARGE_M = 1000  # Big M value for conditional constraints
NUM_PLANES = 10  # Define the number of aircrafts

# Define arrival times and costs for each aircraft
earliest_arrival = [129, 195, 89, 96, 110, 120, 124, 126, 135, 160]  # Earliest arrival times
scheduled_arrival = [155, 258, 96, 106, 123, 135, 138, 140, 150, 180]  # Scheduled arrival times
latest_arrival = [559, 744, 510, 521, 555, 576, 577, 573, 591, 657]  # Latest arrival times
early_penalty = [10, 10, 30, 30, 30, 30, 30, 30, 30, 30]  # Penalty for early arrival
late_penalty = [10, 10, 30, 30, 30, 30, 30, 30, 30, 30]  # Penalty for late arrival

# Minimum time between landings for each pair of aircraft
min_gap = [
    [LARGE_M, 3, 15, 15, 15, 15, 15, 15, 15, 15],
    [3, LARGE_M, 15, 15, 15, 15, 15, 15, 15, 15],
    [15, 15, LARGE_M, 8, 8, 8, 8, 8, 8, 8],
    [15, 15, 8, LARGE_M, 8, 8, 8, 8, 8, 8],
    [15, 15, 8, 8, LARGE_M, 8, 8, 8, 8, 8],
    [15, 15, 8, 8, 8, LARGE_M, 8, 8, 8, 8],
    [15, 15, 8, 8, 8, 8, LARGE_M, 8, 8, 8],
    [15, 15, 8, 8, 8, 8, 8, LARGE_M, 8, 8],
    [15, 15, 8, 8, 8, 8, 8, 8, LARGE_M, 8],
    [15, 15, 8, 8, 8, 8, 8, 8, 8, LARGE_M],
]

<h3><u>Building the Model</h3></u>

We create the optimization model and define the variables using Markov Decision Process (MDP) model. This includes four types of variables: penalties for early and late arrivals, and decision variables to determine the landing order and times.


In [107]:
# Create optimization model
model = gp.Model("Aircraft Landing Scheduling")

In [108]:
# Add decision variables for landing times and penalties
landing_times = model.addVars(NUM_PLANES, vtype=GRB.CONTINUOUS, name="landing_time")
early_fines = model.addVars(NUM_PLANES, vtype=GRB.CONTINUOUS, name="early_fine")
late_fines = model.addVars(NUM_PLANES, vtype=GRB.CONTINUOUS, name="late_fine")
landing_sequence = model.addVars(NUM_PLANES, NUM_PLANES, vtype=GRB.BINARY, name="landing_sequence")

<h3><u>Setting Constraints</h3></u>

We define the necessary constraints to ensure feasibility. Planes must land within their designated time windows, maintain the minimum required time gap between landings, and penalties must be non-negative.


In [109]:
# Add constraints for landing within the time windows
for plane in range(NUM_PLANES):
    model.addConstr(landing_times[plane] >= earliest_arrival[plane], name=f"earliest_{plane}")
    model.addConstr(landing_times[plane] <= latest_arrival[plane], name=f"latest_{plane}")

# Add constraints for minimum time between landings
for i in range(NUM_PLANES):
    for j in range(i + 1, NUM_PLANES):
        model.addConstr(landing_times[i] + min_gap[i][j] <= landing_times[j] + LARGE_M * (1 - landing_sequence[i, j]), name=f"gap_{i}_{j}_1")
        model.addConstr(landing_times[j] + min_gap[j][i] <= landing_times[i] + LARGE_M * landing_sequence[i, j], name=f"gap_{i}_{j}_2")

# Add constraints for penalties
for plane in range(NUM_PLANES):
    model.addConstr(early_fines[plane] >= scheduled_arrival[plane] - landing_times[plane], name=f"early_fine_{plane}")
    model.addConstr(early_fines[plane] >= 0, name=f"non_neg_early_{plane}")
    model.addConstr(late_fines[plane] >= landing_times[plane] - scheduled_arrival[plane], name=f"late_fine_{plane}")
    model.addConstr(late_fines[plane] >= 0, name=f"non_neg_late_{plane}")

<h3><u>Objective Function Definition</u></h3>

The objective function is defined to minimize the total penalty cost. It computes the total penalty for each plane, considering both early and late arrivals, and aims to minimize this sum.


In [110]:
# Set the objective function to minimize the total penalty
model.setObjective(gp.quicksum(early_penalty[plane] * early_fines[plane] + late_penalty[plane] * late_fines[plane] for plane in range(NUM_PLANES)), GRB.MINIMIZE)

<h3><u>Running the Optimization</u></h3>

The `optimize` method initiates the optimization process, finding the best landing times that minimize the total penalty while satisfying all constraints.



In [111]:
# Optimize the model
model.optimize()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 7 3700U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 150 rows, 130 columns and 350 nonzeros
Model fingerprint: 0xab16ea26
Variable types: 30 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+01, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+03]
Found heuristic solution: objective 5570.0000000
Presolve removed 50 rows and 55 columns
Presolve time: 0.00s
Presolved: 100 rows, 75 columns, 300 nonzeros
Variable types: 30 continuous, 45 integer (45 binary)

Root relaxation: objective 0.000000e+00, 41 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node

<h3><u>Displaying Results</u></h3>

Using the `tabulate` library, we print a table showing the scheduled landing time for each plane, ensuring the total penalty is minimized.



In [112]:
# Output results with the scheduled landing time for each aircraft and the minimum total penalty
if model.status == GRB.OPTIMAL:
    landing_schedule = model.getAttr('x', landing_times)   # Extract the scheduled landing times for each aircraft
    # Prepare the results table
    results = []
    for plane in range(NUM_PLANES):
        results.append([f"Aircraft {plane + 1}", f"{landing_schedule[plane]:4.0f} minutes"])
    print(tabulate(results, headers=["AIRCRAFT", "SCHEDULED LANDING TIME"], tablefmt="pipe"))  # Print the results table
    print(f"Minimum total penalty: {model.objVal}")  # Print the minimum total penalty
else:
    print("Optimal solution not found!")

| AIRCRAFT    | SCHEDULED LANDING TIME   |
|:------------|:-------------------------|
| Aircraft 1  | 165 minutes              |
| Aircraft 2  | 258 minutes              |
| Aircraft 3  | 96 minutes               |
| Aircraft 4  | 106 minutes              |
| Aircraft 5  | 118 minutes              |
| Aircraft 6  | 126 minutes              |
| Aircraft 7  | 134 minutes              |
| Aircraft 8  | 142 minutes              |
| Aircraft 9  | 150 minutes              |
| Aircraft 10 | 180 minutes              |
Minimum total penalty: 700.0


<h1><u>2nd Problem: Oil delivery planning</h1></u>

<h3><u>Defining Input Parameters</h3></u>

This section initializes the input data as specified in the problem statement.

In [113]:
# Define the city locations
cities = ['Ω','Α','Β','Γ','Δ','Ε','ΣΤ'] 

# Define the demand for oil in each city
demand = [0,14000,3000,6000,16000,15000,5000]

# Define the distances between cities in kilometers
distances = [[0, 148, 55, 32, 70, 140, 73],
             [148, 0, 93, 180, 99, 12, 72],
             [55, 93, 0, 85, 20, 83, 28],
             [32, 180, 85, 0, 100, 174, 99],
             [70, 99, 20, 100, 0, 85, 49],
             [140, 12, 83, 174, 85, 0, 73],
             [73, 72, 28, 99, 49, 73, 0]] 

# Define the capacity of the oil transporter
capacity = 39000

<h3><u>Building the Model</h3></u>

In [114]:
# Create a Gurobi optimization model
model = gp.Model("oil_transport") 

# Get the total number of cities
num_locations = len(cities) 

# Add binary decision variables to represent whether oil is transported between cities
transports = model.addVars(num_locations, num_locations, vtype=GRB.BINARY, name='transports')  
# Add integer decision variables to represent the sequence of city visits in the transportation route
transport_sequence = model.addVars(num_locations, vtype=GRB.INTEGER, name='transport_sequence')

<h3><u>Setting Constraints</h3></u>

In [115]:
# Add constraint: total demand at the start must not exceed transporter's capacity
model.addConstr(gp.quicksum(demand[j] * transports[0, j] for j in range(1, num_locations)) <= capacity, name="capacity")

# Add constraint: the first city in the sequence of visited cities must be the city with index 0
model.addConstr(transport_sequence[0] == 0)

# Add constraints to ensure each location is visited exactly once
for j in range(1, num_locations):
    model.addConstr(gp.quicksum(transports[i, j] for i in range(num_locations) if i != j) == 1)
    model.addConstr(gp.quicksum(transports[j, k] for k in range(num_locations) if k != j) == 1)

# Prevent subtours by ensuring each route includes the city O
for i in range(1, num_locations):
    for j in range(1, num_locations):
        if i != j:
            model.addConstr(transport_sequence[i] - transport_sequence[j] + capacity * transports[i, j] <= capacity - demand[j], name=f"subtour_{i}_{j}")

# Load after delivering each city must be at least their demand and within transporter's capacity
for i in range(1, num_locations):
    model.addConstr(transport_sequence[i] >= demand[i])
    model.addConstr(transport_sequence[i] <= capacity) 

<h3><u>Objective Function Definition</u></h3>


In [116]:
# Define the objective function: minimize the total distance
model.setObjective(gp.quicksum(distances[i][j] * transports[i, j] for i in range(num_locations) for j in range(num_locations) if i != j), GRB.MINIMIZE)

<h3><u>Running the Optimization</u></h3>

In [117]:
# Optimize the model
model.optimize()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 7 3700U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 56 rows, 56 columns and 181 nonzeros
Model fingerprint: 0xbfbc67ad
Variable types: 0 continuous, 56 integer (49 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+04]
  Objective range  [1e+01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+04]
Found heuristic solution: objective 681.0000000
Presolve removed 13 rows and 14 columns
Presolve time: 0.00s
Presolved: 43 rows, 42 columns, 162 nonzeros
Variable types: 0 continuous, 42 integer (36 binary)

Root relaxation: objective 2.461282e+02, 12 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

 

<h3><u>Displaying Results</u></h3>

In [118]:
# Output results
if model.status == GRB.OPTIMAL:
    print('Minimum distance: %g km' % model.objVal)  # Print the minimum distance achieved by the optimized model
    best_routes = model.getAttr('x', transports)

    # Create lists to store routes and a table for printing
    array_of_routes = []  # Empty list to store route indices
    route_table = []  # Empty list to store routes for printing
    
    # Extract best routes
    for i in range(num_locations):
        for j in range(num_locations):
            if i != j and best_routes[i, j] > 0.5:  # Check if a route exists between cities i and j
                route_table.append((cities[i], cities[j]))  
                array_of_routes.append((i, j))

    print(tabulate(route_table, headers=["From City", "Destination"], tablefmt="pipe"))  

    # Define a function to find a route starting from a given city
    def find_route(start, remaining_routes): 
        route = [start]  # Start the route with the given city
        while True:
            current_route = False
            for idx, route_connection in enumerate(remaining_routes):
                if route_connection[0] == start:  # Check if the route starts from the current city
                    route.append(route_connection[1])  
                    start = route_connection[1]  
                    del remaining_routes[idx]  
                    current_route = True
                    break
            if not current_route:
                break
        return route  

    routes_to_process = array_of_routes.copy() 
    found_routes = []  

    # Find all routes starting from each city
    while routes_to_process: 
        starting_city = routes_to_process[0][0]  
        route = find_route(starting_city, routes_to_process)  
        found_routes.append(route)  

    print('Best sequence of visiting the cities:') 
    for route in found_routes:
        city_sequence = [cities[i] for i in route]  
        print(" -> ".join(city_sequence))  # Print route in the desired format

    print('Best sequence of visiting the cities in order of routes:') 
    for route in route_table:
        print("Route from", route[0], "to", route[1])  # Print route in the desired format

else:
    print("No optimal solution found!")


Minimum distance: 497 km
| From City   | Destination   |
|:------------|:--------------|
| Ω           | Β             |
| Ω           | Γ             |
| Α           | ΣΤ            |
| Β           | Ε             |
| Γ           | Δ             |
| Δ           | Ω             |
| Ε           | Α             |
| ΣΤ          | Ω             |
Best sequence of visiting the cities:
Ω -> Β -> Ε -> Α -> ΣΤ -> Ω -> Γ -> Δ -> Ω
Best sequence of visiting the cities in order of routes:
Route from Ω to Β
Route from Ω to Γ
Route from Α to ΣΤ
Route from Β to Ε
Route from Γ to Δ
Route from Δ to Ω
Route from Ε to Α
Route from ΣΤ to Ω
